# Illiquidity Calculation - Trade-by-Trade

  - This notebook walks through illiquidity calculations based on methodology in The Illiquidity of Corporate Bonds, Bao, Pan, and Wang (2010).

  - In order to avoid re-running the notebook every time it changes (it changes often, even by the act of opening it) and to only rerun it if meaningful changes have been made, the build system only looks for changes in the plaintext version of the notebook. That is, the notebook is converted to a Python script via [nbconvert](https://nbconvert.readthedocs.io/en/latest/), which is often packaged with Jupyter.
  Then, DoIt looks for changes to the Python version. If it detects a difference, then the notebook is re-run. (Note, that you could also convert to a Markdown file with 
  [JupyText](https://github.com/mwouts/jupytext). However, this package is often not packaged with Jupyter.)
  - Since we want to use Jupyter Notebooks for exploratory reports, we want to keep fully-computed versions of the notebook (with the output intact). However, earlier I said that I strip the notebook of its output before committing to version control. Well, to keep the output, every time PyDoit runs the notebook, it outputs an HTML version of the freshly run notebook and saves that HTML report in the `output` directory. That way, you will be able to view the finished report at any time without having to open Jupyter.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import config

OUTPUT_DIR = config.OUTPUT_DIR
DATA_DIR = config.DATA_DIR

In [2]:
import table2_calc_illiquidity

# Step 1: Clean Merged Data for Intraday Illiquidity Calculation

Before calculating illiquidity measures, it's essential to ensure that our corporate bond data is accurate and relevant. The `clean_intraday` function takes care of preparing the intraday data by performing several critical cleaning steps:

- Merges trade data and trade time to timestamp
- Runs Dickerson filters to remove trades that the per-filtering steps missed


In [3]:
df = table2_calc_illiquidity.clean_intraday('2003-04-14', '2009-06-30')
df.head()

,cusip,trd_exctn_dt,trd_exctn_tm,days_to_sttl_ct,lckd_in_ind,wis_fl,msg_seq_nb,entrd_vol_qt,prclean,orig_msg_seq_nb,trd_tmstamp,month_year
4187316,001546AE0,2003-04-14,12:05:33,000,None,N,0010675,5000000.0,94.3750,None,2003-04-14 12:05:33,2003-04
4187317,001546AE0,2003-04-14,13:00:16,000,None,N,0014878,1000000.0,93.6250,None,2003-04-14 13:00:16,2003-04
4187318,001546AE0,2003-04-14,13:00:44,000,None,N,0014891,1000000.0,93.9375,None,2003-04-14 13:00:44,2003-04
4187319,001546AE0,2003-04-14,13:07:28,000,None,N,0015340,2000000.0,93.5000,None,2003-04-14 13:07:28,2003-04
4187320,001546AE0,2003-04-14,13:07:52,000,None,N,0015352,1000000.0,94.0000,None,2003-04-14 13:07:52,2003-04


# Step 2: Calculate Price Changes and Perform Additional Cleaning

In this part of the analysis pipeline, we use the `calc_deltaprc` function to compute intraday price changes for corporate bonds, designed to operate on cleaned trade-by-trade corporate bond trade data.

This calculation is based on the Measure of Illiquidity on page 10 and 11 of the peper: $ \gamma = -\text{Cov}(p_t - p_{t-1}, p_{t+1} - p_t) $. The process involves several steps:
- Calculation of Log Prices: Transform cleaned prices to log prices for more stable numerical properties.
- Lagged and Lead Price Changes: Determine the price changes by computing lagged and lead log prices.
- Restricting Returns: Ensure that calculated price changes (returns) are within the range of -100% to 100%.
- Conversion to Percentage: Change the representation of price changes from decimal to percentage for clarity.
- Cleaning Data: Remove entries with incomplete information to maintain the quality of the dataset.
- Filtering by Trade Count: Exclude bonds with fewer than 10 trade observations to focus on more reliable data.

This function is essential for preparing the bond price data for accurate calculation of financial metrics such as illiquidity.

In [4]:
df = table2_calc_illiquidity.calc_deltaprc(df)
df.head()

,cusip,trd_exctn_dt,trd_exctn_tm,days_to_sttl_ct,lckd_in_ind,wis_fl,msg_seq_nb,entrd_vol_qt,prclean,orig_msg_seq_nb,trd_tmstamp,month_year,logprc,logprc_lag,deltap,logprc_lead,deltap_lag
4187317,001546AE0,2003-04-14,13:00:16,000,None,N,0014878,1000000.0,93.6250,None,2003-04-14 13:00:16,2003-04,4.539297,4.547276,-0.797877,4.542630,0.333223
4187318,001546AE0,2003-04-14,13:00:44,000,None,N,0014891,1000000.0,93.9375,None,2003-04-14 13:00:44,2003-04,4.542630,4.539297,0.333223,4.537961,-0.466823
4187319,001546AE0,2003-04-14,13:07:28,000,None,N,0015340,2000000.0,93.5000,None,2003-04-14 13:07:28,2003-04,4.537961,4.542630,-0.466823,4.543295,0.533335
4187320,001546AE0,2003-04-14,13:07:52,000,None,N,0015352,1000000.0,94.0000,None,2003-04-14 13:07:52,2003-04,4.543295,4.537961,0.533335,4.547276,0.398143
4187321,001546AE0,2003-04-14,13:10:53,000,None,N,0019051,1000000.0,94.3750,None,2003-04-14 13:10:53,2003-04,4.547276,4.543295,0.398143,4.546614,-0.066247


# Step 3: Annual Illiquidity Metrics Calculation

This step involves using the `calc_annual_illiquidity_table` function to calculate and summarize annual illiquidity metrics for corporate bonds. The function takes intraday bond data as input and computes several statistics that capture the illiquidity of bonds on an annual basis.

- Computes the illiquidity for each bond and month by taking the negative of the covariance between intraday price changes (`deltap`) and their lagged values (`deltap_lag`).

- Aggregated the monthly illiquidity measures to obtain annual statistics, including mean and median illiquidity.

- Calculates t-statistics for the mean illiquidity of each bond and year and determines the percentage of these t-stats that are significant (>= 1.96).

- Calculates robust t-stats are calculated using OLS with HAC (heteroskedasticity and autocorrelation consistent) standard errors.

- Calculate overall statistics across the full sample period.

- Compiles all these metrics into a table that presents the mean and median illiquidity, the percentage of significant t-statistics, and robust t-statistics for each year, as well as for the full sample period.

This comprehensive illiquidity metric calculation allows us to understand the annual and overall liquidity characteristics of the corporate bond market.

In [5]:
_, table2_intraday = table2_calc_illiquidity.calc_annual_illiquidity_table(df)
table2_intraday

  0%|          | 0/53737 [00:00<?, ?it/s]

  0%|          | 1/53737 [00:00<11:01:59,  1.35it/s]

  1%|          | 477/53737 [00:00<01:08, 772.08it/s]

  2%|▏         | 907/53737 [00:00<00:36, 1434.15it/s]

  3%|▎         | 1384/53737 [00:01<00:24, 2138.33it/s]

...

 98%|█████████▊| 52404/53737 [00:09<00:00, 6040.37it/s]

 99%|█████████▊| 53010/53737 [00:09<00:00, 6024.28it/s]

100%|█████████▉| 53644/53737 [00:09<00:00, 6108.85it/s]

100%|██████████| 53737/53737 [00:09<00:00, 5623.28it/s]

,Year,Mean illiq,Median illiq,Per t greater 1.96,Robust t stat
0,2003,1.303793,0.337898,88.762122,55.723498
1,2004,1.019589,0.213466,89.986712,1.274378
2,2005,0.665460,0.142755,93.402677,72.538561
3,2006,0.371456,0.119556,93.900834,2.430898
4,2007,0.278741,0.115967,93.330687,2.180491
5,2008,2.121446,0.259996,81.552785,31.901778
6,2009,2.401927,0.363650,88.403179,36.825191
7,Full,0.973140,0.186082,90.721015,3.154765
